In [ ]:
from sqlalchemy.engine import create_engine
from sqlalchemy import inspect
from sqlalchemy import text
from pprint import pprint

import csv

mmCourse = 'lgdm'

sqlite_engine = create_engine('sqlite:////Users/saisun/ProjectCode/uqx_tools/micromastersreports/databases/' + mmCourse + '_verified.db')
sqlite_connection = sqlite_engine.connect()

mmCoursesMeta = {
    'buslead': {
        "buslead1x": {"title": "UQx_BUSLEAD1x",  'grade': 0.65},
        "buslead2x": {"title": "UQx_BUSLEAD2x",  'grade': 0.65},
        "buslead3x": {"title": "UQx_BUSLEAD3x",  'grade': 0.65},
        "buslead4x": {"title": "UQx_BUSLEAD4x",  'grade': 0.65},
        "buslead5x": {"title": "UQx_BUSLEAD5x",  'grade': 0.65},
    },
    'corpinn': {
        "corpinn1x": {"title": "UQx_CORPINN1x", 'grade': 0.7},
        "corpinn2x": {"title": "UQx_CORPINN2x",  'grade': 0.7},
        "corpinn3x": {"title": "UQx_CORPINN3x",  'grade': 0.7},
        "corpinn4x": {"title": "UQx_CORPINN4x",  'grade': 0.7},
        "corpinn5x": {"title": "UQx_CORPINN5x",  'grade': 0.5},
    },
    'engy': {
        "engy0x": {"title": "UQx_ENGY0x", 'grade': 0.7},
        "engy1x": {"title": "UQx_ENGY1x", 'grade': 0.7},
        "engy2x": {"title": "UQx_ENGY2x", 'grade': 0.7},
        "engy3x": {"title": "UQx_ENGY3x", 'grade': 0.7},
        "engycapx": {"title": "UQx_ENGYCAPx", 'grade': 0.7},
    },
    'lgdm': {
        "lgdm1x": {"title": "UQx_LGDM1x", 'grade': 0.65},
        "lgdm2x": {"title": "UQx_LGDM2x",  'grade': 0.65},
        "lgdm3x": {"title": "UQx_LGDM3x",  'grade': 0.65},
        "lgdm4x": {"title": "UQx_LGDM4x",  'grade': 0.65},
        "lgdm5x": {"title": "UQx_LGDM5x",  'grade': 0.65},
    },
}

micromasters_courses = mmCoursesMeta[mmCourse]

def update_course_instancedetails_auditpassed():
    print('update_course_instancedetails_auditpassed')
    ci_query = 'SELECT * FROM course_instancedetails'
    result = sqlite_connection.execute(ci_query)
    for row in result:
        update_id = int(row[0])
        course = row[6]
        pass_line = micromasters_courses[course]['grade']
        #print('pass_line', pass_line)
        grade = row[2]
        enrollment_track = row[3]
        update_ci = "UPDATE course_instancedetails SET audit_passed='%s' WHERE id=%d; " %('N', update_id)
        if (enrollment_track == 'audit') and (grade >= pass_line):
            update_ci = "UPDATE course_instancedetails SET audit_passed='%s' WHERE id=%d; " %('Y', update_id)
        sqlite_connection.execute(update_ci)
        
        
def add_last_run():
    for course in micromasters_courses:
        query = "SELECT course_instance FROM course_instancedetails WHERE course='%s' ORDER BY year DESC, term DESC;" % (course)
        query_result = sqlite_connection.execute(query).fetchone()
        if query_result is not None:
            micromasters_courses[course]['last_run'] = query_result[0]
    #pprint(micromasters_courses)
    

def find_audit_passed(course, pointed_run=''):
    email_list = []
    if pointed_run == '':
        pointed_run = micromasters_courses[course]['last_run']
    query = "SELECT c.learner_id, c.course_instance, c.audit_passed, l.email FROM course_instancedetails c, learners l WHERE c.learner_id = l.student_id AND c.course_instance = '%s' AND c.audit_passed = 'Y'" % (pointed_run)
    #print(query)
    query_result = sqlite_connection.execute(query)
    for record in query_result:
        email_list.append(record['email'])
    auditpassed_emails[course] = email_list
        
auditpassed_emails = {}   
pointed_course_instances = {
    'lgdm1x': 'UQx_LGDM1x_1T2017',
    'lgdm2x': 'UQx_LGDM2x_2T2017',
    'lgdm3x': 'UQx_LGDM3x_1T2018',
    'lgdm4x': 'UQx_LGDM4x_1T2018',
}
    
update_course_instancedetails_auditpassed()    
add_last_run()
for course in pointed_course_instances: 
    #find_audit_passed(course)
    find_audit_passed(course, pointed_course_instances[course])

pprint(auditpassed_emails)
print('The End.')

